In [2]:
import requests
import pandas as pd
from urllib.request import Request, urlopen
from sodapy import Socrata
import json
import re
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import ipywidgets as wd
from ipywidgets import interact
import seaborn as sns
#import ptitprince as pt
import base64

In [3]:
with open("newSecret.json", "r") as f:
    config = json.load(f)

token = config["TOKEN_API_OD"]
user=config["user"]
passwd=config["pw"]
client_id=config["clientId"]
client_secret=config["clientSecret"]

## MAIN ACCESS FUNCTION

In [7]:
def get_access_token(client_id, client_secret):
    # URL para obtener el token de acceso
    auth_url = 'https://accounts.spotify.com/api/token'
    
    # Creación del encabezado de autorización con client_id y client_secret codificados en base64
    auth_header = {
        'Authorization': f'Basic {base64.b64encode((client_id + ":" + client_secret).encode()).decode()}'
    }
    
    # Datos necesarios para la petición POST
    auth_data = {
        'grant_type': 'client_credentials',
        'client_id': client_id,
        'client_secret': client_secret
    }
    
    # Realizar la petición POST
    auth_response = requests.post(auth_url, data=auth_data, headers=auth_header)
    
    # Extraer la respuesta y obtener el token de acceso
    auth_response_data = auth_response.json()
    access_token = auth_response_data['access_token']
    
    # Devolver el token de acceso
    return access_token


## CALL ME BY YOUR NAME

In [5]:
def get_artists_by_name(query, access_token, limit=50):
    search_url = 'https://api.spotify.com/v1/search'
    
    headers = {
        'Authorization': f'Bearer {access_token}'
    }
    
    params = {
        'q': f"name:\"{query}\"",
        'type': 'artist',
        'limit': limit
    }
    
    response = requests.get(search_url, headers=headers, params=params)
    response_data = response.json()
    
    if 'artists' in response_data:
        return [artist['id'] for artist in response_data['artists']['items']]
    else:
        print("Error en la respuesta de la API:", response_data)
        return []

In [9]:
access_token = get_access_token(client_id, client_secret)

In [11]:
def extract_artists_to_dataframe(response_data):
    # Lista para almacenar datos de artistas
    artist_list = []

    # Iterar sobre cada artista en la respuesta
    for artist in response_data['artists']:
        # Extraer la información relevante
        artist_info = {
            'Name': artist['name'],
            'Genres': ', '.join(artist['genres']),
            'Popularity': artist['popularity'],
            'Spotify_URL': artist['external_urls']['spotify'],
            'Followers': artist['followers']['total'],
            'Image_URL': artist['images'][0]['url'] if artist['images'] else None, # Toma la primera imagen, si existe
            'Artist_ID': artist['id'],
            'Artist_Type': artist['type'],
            'Artist_URI': artist['uri']
        }
        # Agregar a la lista
        artist_list.append(artist_info)

    # Convertir lista en DataFrame
    df_artists = pd.DataFrame(artist_list)

    return df_artists

In [13]:
def get_artists_from_spotify(token, artist_ids):
    """
    Fetch artist details from Spotify's API.
    
    :param access_token: Token for API authorization.
    :param artist_ids: List of artist IDs.
    :return: Response data as a dictionary.
    """
    
    # Endpoint de la API
    base_url = "https://api.spotify.com/v1/artists"
    
    # Headers de la petición (incluye el token de acceso para autorización)
    headers = {
        "Authorization": f"Bearer {token}"
    }
    
    # Parámetros de la petición (IDs de los artistas)
    params = {
        "ids": ",".join(artist_ids)
    }
    
    # Realizar la petición GET a la API de Spotify
    response = requests.get(base_url, headers=headers, params=params)
    
    # Convertir la respuesta en formato JSON a un diccionario de Python
    response_data = response.json()

    return response_data

""" Ejemplo de uso:

artist_ids = ["2CIMQHirSU0MQqyYHq0eOx", "57dN52uHvrHOxijzpIgu3E"]  # Reemplaza con los IDs de los artistas que quieras consultar
token=access_token
 Obtener respuesta de la API
response_data = get_artists_from_spotify(token, artist_ids)

# Procesar la respuesta con la función extract_artists_to_dataframe
df_artist = extract_artists_to_dataframe(response_data)
#print(df)
df_artist"""

' Ejemplo de uso:\n\nartist_ids = ["2CIMQHirSU0MQqyYHq0eOx", "57dN52uHvrHOxijzpIgu3E"]  # Reemplaza con los IDs de los artistas que quieras consultar\ntoken=access_token\n Obtener respuesta de la API\nresponse_data = get_artists_from_spotify(token, artist_ids)\n\n# Procesar la respuesta con la función extract_artists_to_dataframe\ndf_artist = extract_artists_to_dataframe(response_data)\n#print(df)\ndf_artist'

## THE INPUT

In [14]:
Spotify_List_Id='7M36AGwbD9OpGUZN3IWx7V'

In [15]:
def get_band_from_playlist_id(token, playlist_id):
    """
    Fetch band names from a Spotify playlist.
    
    :param token: Token for API authorization.
    :param playlist_id: Spotify Playlist ID.
    :return: List of band names.
    """
    playlist_url = f"https://api.spotify.com/v1/playlists/{playlist_id}/tracks"
    headers = {"Authorization": f"Bearer {token}"}
    response = requests.get(playlist_url, headers=headers)
    
    if response.status_code == 200:
        playlist_data = response.json()
        bands = []
        for item in playlist_data['items']:
            # Cada 'item' contiene información de una pista, incluyendo sus artistas
            track_artists = item['track']['artists']
            for artist in track_artists:
                bands.append(artist['name'])
        return bands
    else:
        print(f"Error fetching playlist data: {response.status_code}")
        return []

# Uso de la función
# token = "tu_token_de_acceso"
# playlist_id = "id_de_tu_playlist"
# band_names = get_band_from_playlist_id(token, playlist_id)
# print(band_names)


In [16]:
band_names = get_band_from_playlist_id(token, Spotify_List_Id)

In [18]:
Band_Names=pd.DataFrame(band_names)
Band_Names

,0
0,Deformation
1,SYRACUSÆ
2,Jose Macario
3,Under Threat
4,Demonsquest
...,...
102,Epiphany.
103,Licantropia
104,ubergehen
105,Cosmology X


In [ ]:
def get_band_from_list_id(token, list_id):
    """
    Fetch band name from an artist from Spotify's API.
    
    :param token: Token for API authorization.
    :param artist_id: Artist ID.
    :return: List of album IDs.
    """
    list_url = f"https://api.spotify.com/v1/playlists/{playlist_id}/tracks"
    headers = {"Authorization": f"Bearer {token}"}
    response = requests.get(list_url, headers=headers)
    
    if response.status_code == 200:
        bands_data = response.json()
        return [band['name'] for band in band_data['items']['artists']]
    else:
        print(f"Error fetching band name: {response.status_code}")
        return []

def get_tracks_from_album_ids(token, album_ids):
    """
    Fetch tracks from album IDs from Spotify's API.
    
    :param token: Token for API authorization.
    :param album_ids: List of album IDs.
    :return: List of track data dictionaries.
    """
    all_tracks = []
    for album_id in album_ids:
        tracks_url = f"https://api.spotify.com/v1/albums/{album_id}/tracks"
        headers = {"Authorization": f"Bearer {token}"}
        response = requests.get(tracks_url, headers=headers)
        
        if response.status_code == 200:
            tracks_data = response.json()
            all_tracks.extend(tracks_data['items'])
        else:
            print(f"Error fetching tracks for album {album_id}: {response.status_code}")
    
    return all_tracks